# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/2_feature_pipeline.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import json

from functions import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)

In [3]:
today = datetime.date.today()
forecast_day = today + datetime.timedelta(days=7)
hindcast_day = today - datetime.timedelta(days=1)

---

## <span style='color:#ff5f27'> 🌫 Filling gaps in Air Quality data (PM2.5)</span>

### First time we will determine the 'last update date' using our backfill data
#### Next time we will use `feature view` method from Hopsworks Feature Store

### <span style='color:#ff5f27'>  🧙🏼‍♂️ Parsing PM2.5 data

In [4]:
start_of_cell = time.time()

df_aq_raw = pd.DataFrame()

for continent in target_cities:
    for city_name, coords in target_cities[continent].items():
        df_ = get_aqi_data_from_open_meteo(city_name=city_name,
                                           coordinates=coords,
                                           start_date=str(hindcast_day),
                                           end_date=str(today))
        df_aq_raw = pd.concat([df_aq_raw, df_]).reset_index(drop=True)
    
end_of_cell = time.time()
print("-" * 64)
print(f"Parsed new PM2.5 data for ALL locations up to {str(today)}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Processed PM2_5 for Amsterdam since 2023-04-25 till 2023-04-26.
Took 4.16 sec.

Processed PM2_5 for Athina since 2023-04-25 till 2023-04-26.
Took 2.36 sec.

Processed PM2_5 for Berlin since 2023-04-25 till 2023-04-26.
Took 1.39 sec.

Processed PM2_5 for Gdansk since 2023-04-25 till 2023-04-26.
Took 5.8 sec.

Processed PM2_5 for Kraków since 2023-04-25 till 2023-04-26.
Took 1.16 sec.

Processed PM2_5 for London since 2023-04-25 till 2023-04-26.
Took 0.96 sec.

Processed PM2_5 for Madrid since 2023-04-25 till 2023-04-26.
Took 3.92 sec.

Processed PM2_5 for Marseille since 2023-04-25 till 2023-04-26.
Took 2.55 sec.

Processed PM2_5 for Milano since 2023-04-25 till 2023-04-26.
Took 7.81 sec.

Processed PM2_5 for München since 2023-04-25 till 2023-04-26.
Took 1.1 sec.

Processed PM2_5 for Napoli since 2023-04-25 till 2023-04-26.
Took 1.67 sec.

Processed PM2_5 for Paris since 2023-04-25 till 2023-04-26.
Took 2.34 sec.

Processed PM2_5 for Sevilla since 2023-04-25 till 2023-04-26.
Took 4.38 

In [13]:
df_aq_raw

,city_name,date,pm2_5
0,Amsterdam,2023-04-25,5.6
1,Amsterdam,2023-04-26,8.6
2,Athina,2023-04-25,13.5
3,Athina,2023-04-26,8.4
4,Berlin,2023-04-25,7.4
...,...,...,...
85,Tacoma-S 36th St,2023-04-26,10.6
86,Tukwila Allentown,2023-04-25,7.6
87,Tukwila Allentown,2023-04-26,11.9
88,Tulalip-Totem Beach Rd,2023-04-25,11.5


In [14]:
df_aq_update = df_aq_raw
df_aq_update

,city_name,date,pm2_5
0,Amsterdam,2023-04-25,5.6
1,Amsterdam,2023-04-26,8.6
2,Athina,2023-04-25,13.5
3,Athina,2023-04-26,8.4
4,Berlin,2023-04-25,7.4
...,...,...,...
85,Tacoma-S 36th St,2023-04-26,10.6
86,Tukwila Allentown,2023-04-25,7.6
87,Tukwila Allentown,2023-04-26,11.9
88,Tulalip-Totem Beach Rd,2023-04-25,11.5


### <span style="color:#ff5f27;">🛠 Feature Engineering PM2.5</span>

In [15]:
df_aq_update['date'] = pd.to_datetime(df_aq_update['date'])

In [16]:
# df_aq_update = feature_engineer_aq(df_aq_update)
df_aq_update = df_aq_update.dropna()

In [17]:
df_aq_update.isna().sum().sum()

0

In [18]:
df_aq_update.shape

(90, 3)

In [19]:
df_aq_update.columns

Index(['city_name', 'date', 'pm2_5'], dtype='object')

---

### <span style='color:#ff5f27'>  🧙🏼‍♂️ Parsing Weather data

In [ ]:
start_of_cell = time.time()

df_weather_update = pd.DataFrame()


for city_name, coords in target_cities["US"].items():
    df_ = get_weather_data_from_open_meteo(city_name=city_name,
                                           coordinates=coords,
                                           start_date=str(today),
                                           end_date=str(forecast_day),
                                           forecast=True)
    df_weather_update = pd.concat([df_weather_update, df_]).reset_index(drop=True)


# for continent in target_cities:
#     for city_name, coords in target_cities[continent].items():
#         df_ = get_weather_data_from_open_meteo(city_name=city_name,
#                                                coordinates=coords,
#                                                start_date=str(today),
#                                                end_date=str(forecast_day),
#                                                forecast=True)
#         df_weather_update = pd.concat([df_weather_update, df_]).reset_index(drop=True)
    
end_of_cell = time.time()
print("-" * 64)
print(f"Parsed new weather data for ALL cities up to {str(today)}.")
print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")

Parsed weather for Albuquerque since 2023-04-26 till 2023-05-03.
Took 1.93 sec.

Parsed weather for Atlanta since 2023-04-26 till 2023-05-03.
Took 1.43 sec.

Parsed weather for Chicago since 2023-04-26 till 2023-05-03.
Took 1.64 sec.

Parsed weather for Columbus since 2023-04-26 till 2023-05-03.
Took 5.4 sec.

Parsed weather for Dallas since 2023-04-26 till 2023-05-03.
Took 2.93 sec.

Parsed weather for Denver since 2023-04-26 till 2023-05-03.
Took 1.6 sec.

Parsed weather for Houston since 2023-04-26 till 2023-05-03.
Took 1.62 sec.

Parsed weather for Los Angeles since 2023-04-26 till 2023-05-03.
Took 1.57 sec.

Parsed weather for New York since 2023-04-26 till 2023-05-03.
Took 1.89 sec.



In [ ]:
df_weather_update

In [ ]:
df_aq_update.date = pd.to_datetime(df_aq_update.date)
df_weather_update.date = pd.to_datetime(df_weather_update.date)

df_aq_update["unix_time"] = df_aq_update["date"].apply(convert_date_to_unix)
df_weather_update["unix_time"] = df_weather_update["date"].apply(convert_date_to_unix)

In [ ]:
df_aq_update.date = df_aq_update.date.astype(str)
df_weather_update.date = df_weather_update.date.astype(str)

---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks


project = hopsworks.login()
fs = project.get_feature_store() 

air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather',
    version = 1
)

In [ ]:
air_quality_fg.insert(df_aq_update, write_options={"wait_for_job": False})

In [ ]:
weather_fg.insert(df_weather_update, write_options={"wait_for_job": False})